# Pandas и большие файлы

In [1]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

# Объединение датафреймов

### Данные со слайдов

In [2]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [3]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [4]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [5]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [6]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [7]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [8]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [9]:
visits_grouped.join(purchases_pivot, how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [10]:
visits_grouped.join(purchases_pivot, how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [11]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [12]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [13]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [14]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [15]:
visits_grouped.join(purchases_pivot, how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки

In [16]:
visits_grouped.join(purchases_pivot, how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


# Конкатенация таблиц

In [17]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [18]:
a

,date,value_a
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3


In [19]:
b

,date,value_b
0,2020-01-01,3
1,2020-01-02,4
2,2020-01-03,5


In [20]:
pd.concat([a, b])

,date,value_a,value_b
0,2020-01-01,1.0,NaN
1,2020-01-02,2.0,NaN
2,2020-01-03,3.0,NaN
0,2020-01-01,NaN,3.0
1,2020-01-02,NaN,4.0
2,2020-01-03,NaN,5.0


In [21]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

,date,value_a,date,value_b
0,2020-01-01,1,2020-01-01,3
1,2020-01-02,2,2020-01-02,4
2,2020-01-03,3,2020-01-03,5


### Дубликаты при объединении таблиц

In [22]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [23]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [24]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [25]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [26]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [27]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [28]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [29]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


### Оптимизация хранения данных

In [30]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [31]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [32]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [33]:
len(ratings)

100004

In [34]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [35]:
len(movies)

9125

In [36]:
len(ratings) == len(joined)

True

In [37]:
joined.to_csv('joined_ratings.csv', index=False)

In [38]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [39]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [40]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

9066

### Какой жанр имеет самые высокие рейтинги?

In [41]:
import numpy as np

In [42]:
genres = ['Drama', 'Action', 'Thriller']

In [43]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [44]:
len(ratings)

100004

In [45]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
len(movies)

9125

In [47]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [48]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [49]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [50]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [51]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

Wall time: 16.5 s


In [52]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [53]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN


### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [54]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [55]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [56]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [57]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4
